In [19]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
from scipy.stats import pearsonr, spearmanr
from scipy import stats
import pingouin as pg

In [3]:
# 데이터 불러오기
df1 = pd.read_csv(
    r"C:\Users\user\Desktop\260218_데이터분석\분석용\Step2_입지변수_결합완료.csv",
    encoding="utf-8-sig"
)

df2 = pd.read_csv(
    r"C:\Users\user\Desktop\260218_데이터분석\수집 후 출력(CSV)\월평균_리뷰_최근1년_log.csv",
    encoding="utf-8-sig"
)

print("df1 shape:", df1.shape)
print("df2 shape:", df2.shape)

df1 shape: (430, 45)
df2 shape: (430, 9)


In [18]:
# 병합 키 설정
merge_key = "place_id"  

In [19]:
# df2에서 필요한 컬럼만 선택 (핵심)
target_col = "log_월평균리뷰수"

df2_small = df2[[merge_key, target_col]].copy()

In [20]:
# df1에 동일 컬럼 존재 시 사전 제거
if target_col in df1.columns:
    df1 = df1.drop(columns=[target_col])

In [21]:
# 병합 전 df2 키 중복 체크 (m:1 강제 전 확인)
dup_check = df2_small[merge_key].duplicated().sum()
print("df2 key 중복 개수:", dup_check)

if dup_check > 0:
    raise ValueError("❌ df2에 동일 place_id 중복 존재 → m:1 병합 불가")

df2 key 중복 개수: 0


In [22]:
# 안전 병합 (절대 중복 방지)
df = pd.merge(
    df1,
    df2_small,
    on=merge_key,
    how="left",
    validate="m:1"
)

print("병합 후 shape:", df.shape)

병합 후 shape: (430, 46)


In [23]:
# 병합 후 중복 컬럼 검사 (강제 차단)
duplicated_cols = df.columns[df.columns.duplicated()]

if len(duplicated_cols) > 0:
    raise ValueError(f"❌ 중복 컬럼 발생: {duplicated_cols}")

print("✅ 중복 컬럼 없음")

✅ 중복 컬럼 없음


In [ ]:
# 성과 변수 정의
성과변수 = [
    "log_방문자_리뷰수",
    "log_블로그_리뷰수",
    "log_월평균리뷰수"
]

# log_방문자_리뷰수 → 오프라인 체험 기반 반응
# log_블로그_리뷰수 → 콘텐츠 확산 효과
# log_월평균리뷰수 → 최근 1년 기준의 지속적 관심/활동성 지표

### **5-1. 사전 가정 검정 (공통)**

In [25]:
# Shapiro-Wilk 정규성 검정
print("===== 전체 표본 정규성 검정 =====")
print("H0: 정규성 만족 / H1: 정규성 위반\n")

for col in 성과변수:
    data = df[col].dropna()

    if len(data) < 3:
        print(f"{col} → 표본 부족")
        continue

    stat, p = shapiro(data)

    print(f"{col}")
    print(f"W={stat:.4f}, p={p:.4e}")

    if p > 0.05:
        print(" → 정규성 만족\n")
    else:
        print(" → 정규성 위반\n")


===== 전체 표본 정규성 검정 =====
H0: 정규성 만족 / H1: 정규성 위반

log_방문자_리뷰수
W=0.7835, p=1.8053e-23
 → 정규성 위반

log_블로그_리뷰수
W=0.9823, p=4.1511e-05
 → 정규성 위반

log_월평균리뷰수
W=0.9652, p=1.4316e-08
 → 정규성 위반



In [26]:
# 브랜드별 정규성 검정
print("===== 브랜드별 정규성 검정 =====")

for brand in df["brand"].unique():

    print(f"\n[{brand}]")

    subset = df[df["brand"] == brand]

    for col in 성과변수:
        data = subset[col].dropna()

        if len(data) < 3:
            print(f"{col} → 표본 부족 (n={len(data)})")
            continue

        stat, p = shapiro(data)

        print(f"{col} → W={stat:.4f}, p={p:.4e}")

===== 브랜드별 정규성 검정 =====

[금별맥주]
log_방문자_리뷰수 → W=0.8840, p=1.9349e-07
log_블로그_리뷰수 → W=0.9771, p=7.0947e-02
log_월평균리뷰수 → W=0.9868, p=4.0236e-01

[역전할머니]
log_방문자_리뷰수 → W=0.7711, p=4.6969e-21
log_블로그_리뷰수 → W=0.9776, p=5.5180e-05
log_월평균리뷰수 → W=0.9519, p=7.3439e-09


In [28]:
# Levene 등분산성 검정
브랜드목록 = df["brand"].unique()

print("===== 등분산성 검정 (Levene) =====")
print("H0: 등분산 충족 / H1: 등분산 위반\n")

if len(브랜드목록) != 2:
    print("⚠ 브랜드가 2개가 아닙니다. 확인 필요")
else:
    group1 = df[df["brand"] == 브랜드목록[0]]
    group2 = df[df["brand"] == 브랜드목록[1]]

    for col in 성과변수:

        data1 = group1[col].dropna()
        data2 = group2[col].dropna()

        if len(data1) < 2 or len(data2) < 2:
            print(f"{col} → 표본 부족")
            continue

        stat, p = levene(data1, data2)

        print(f"{col}")
        print(f"W={stat:.4f}, p={p:.4e}")

        if p > 0.05:
            print(" → 등분산 충족\n")
        else:
            print(" → 등분산 위반 (Welch t-test 사용)\n")

===== 등분산성 검정 (Levene) =====
H0: 등분산 충족 / H1: 등분산 위반

log_방문자_리뷰수
W=2.9502, p=8.6592e-02
 → 등분산 충족

log_블로그_리뷰수
W=5.3910, p=2.0710e-02
 → 등분산 위반 (Welch t-test 사용)

log_월평균리뷰수
W=0.8297, p=3.6288e-01
 → 등분산 충족



In [30]:
# 자사 더미 생성
my_brand = "금별맥주"   # 실제 브랜드명으로 수정
df["자사여부"] = (df["brand"] == my_brand).astype(int)

# 입지 그룹 컬럼명 지정
location_col = "역_거리_구분"   # 실제 컬럼명으로 수정

df["입지내_자사매장수"] = (
    df.groupby(location_col)["자사여부"]
      .transform("sum")
)

df["입지내_경쟁매장수"] = (
    df.groupby(location_col)["자사여부"]
      .transform(lambda x: len(x) - x.sum())
)

print("자사/경쟁사 변수 생성 완료")

자사/경쟁사 변수 생성 완료


In [ ]:
df.groupby("역_거리_구분")[["입지내_자사매장수","입지내_경쟁매장수"]].first()

,입지내_자사매장수,입지내_경쟁매장수
역_거리_구분,,
근접,56,213
비근접,47,114


### **절대 규모 해석**
- ***근접 입지(역 근접)***
    - 상권 밀집도가 매우 높음
    - 경쟁 강도 강함
    - **경쟁사 비율: 213/(56+213)=79.2%**
    - **자사 비율 약 20.8%**
    - 전략적 의미
        - 수요는 크지만
        - 경쟁 밀도도 높음
        - 마케팅/브랜드 파워 중요
- ***비근접 입지***
    - 근접 지역보다 전체 매장 수 적음
    - 경쟁 강도 상대적으로 낮음
    - **경쟁사 비율: 114/(47+114)=70.8%**
    - **자사 비율 약 29.2%**
    - 전략적 의미
        - 경쟁이 덜 치열
        - 점유율 확보 전략 가능

### **5-2. 자사 vs 경쟁사 마케팅 성과 직접 비교**

In [36]:
# 집단 정의
my_brand = "금별맥주"   # 자사 브랜드명 정확히 입력

group_self = df[df["brand"] == my_brand]
group_comp = df[df["brand"] != my_brand]

print("자사 표본 수:", len(group_self))
print("경쟁사 표본 수:", len(group_comp))

자사 표본 수: 103
경쟁사 표본 수: 327


In [37]:
# 비교할 성과 변수 정의
성과변수 = [
    "log_방문자_리뷰수",
    "log_블로그_리뷰수",
    "log_월평균리뷰수"
]

In [51]:
# 자동 검정 함수
def run_test(var):

    print(f"📌 변수: {var}")

    data1 = group_self[var].dropna()
    data2 = group_comp[var].dropna()

    if len(data1) < 3 or len(data2) < 3:
        print("표본 부족")
        return

    # 정규성
    stat1, p1 = shapiro(data1)
    stat2, p2 = shapiro(data2)

    normal1 = p1 > 0.05
    normal2 = p2 > 0.05

    # 등분산성
    stat_levene, p_levene = levene(data1, data2)
    equal_var = p_levene > 0.05

    print(f"자사 정규성 p = {p1:.4f}")
    print(f"경쟁사 정규성 p = {p2:.4f}")
    print(f"등분산성 p = {p_levene:.4f}")
    
    # 검정 선택
    if normal1 and normal2:

        if equal_var:
            print("→ 독립표본 t-test")
            stat, p = ttest_ind(data1, data2, equal_var=True)
        else:
            print("→ Welch t-test")
            stat, p = ttest_ind(data1, data2, equal_var=False)

    else:
        print("→ Mann-Whitney U test")
        stat, p = mannwhitneyu(data1, data2, alternative="two-sided")

    print(f"\n검정통계량={stat:.4f}, p-value={p:.4e}")

    if p < 0.05:
        print("✅ 유의한 차이 있음")
    else:
        print("❌ 유의한 차이 없음")
    print("=" * 50)

In [52]:
# 실행
for var in 성과변수:
    if var in df.columns:
        run_test(var)
    else:
        print(f"{var} → 컬럼 없음")

📌 변수: log_방문자_리뷰수
자사 정규성 p = 0.0000
경쟁사 정규성 p = 0.0000
등분산성 p = 0.0866
→ Mann-Whitney U test

검정통계량=20577.5000, p-value=6.8040e-04
✅ 유의한 차이 있음
📌 변수: log_블로그_리뷰수
자사 정규성 p = 0.0709
경쟁사 정규성 p = 0.0001
등분산성 p = 0.0207
→ Mann-Whitney U test

검정통계량=24334.0000, p-value=9.5241e-12
✅ 유의한 차이 있음
📌 변수: log_월평균리뷰수
자사 정규성 p = 0.4024
경쟁사 정규성 p = 0.0000
등분산성 p = 0.3629
→ Mann-Whitney U test

검정통계량=20449.5000, p-value=1.0347e-03
✅ 유의한 차이 있음


### **5-3. 동일 입지 조건 내 자사 vs 경쟁사 비교**

In [54]:
my_brand = "금별맥주"   # 자사 브랜드명

location_col = "역_거리_구분"  # 입지 통제 변수

성과변수 = [
    "log_방문자_리뷰수",
    "log_블로그_리뷰수",
    "log_월평균리뷰수"
]

In [57]:
def stratified_test(sub_df, var):

    group_self = sub_df[sub_df["brand"] == my_brand][var].dropna()
    group_comp = sub_df[sub_df["brand"] != my_brand][var].dropna()

    if len(group_self) < 3 or len(group_comp) < 3:
        print("   표본 부족")
        return

    # 정규성
    p1 = shapiro(group_self)[1]
    p2 = shapiro(group_comp)[1]

    normal = (p1 > 0.05) and (p2 > 0.05)

    # 등분산
    p_levene = levene(group_self, group_comp)[1]
    equal_var = p_levene > 0.05

    # 검정 선택
    if normal:
        if equal_var:
            stat, p = ttest_ind(group_self, group_comp, equal_var=True)
            method = "독립표본 t-test"
        else:
            stat, p = ttest_ind(group_self, group_comp, equal_var=False)
            method = "Welch t-test"
    else:
        stat, p = mannwhitneyu(group_self, group_comp, alternative="two-sided")
        method = "Mann-Whitney U"

    print(f"   방법: {method}")
    print(f"   p-value: {p:.4e}")

    if p < 0.05:
        print("   ✅ 동일 입지 내에서도 브랜드 차이 존재")
    else:
        print("   ❌ 동일 입지 내 브랜드 차이 없음")
    print("=" * 50 )

In [59]:
for loc in df[location_col].unique():

    print(f"📍 입지 그룹: {loc}")

    subset = df[df[location_col] == loc]

    for var in 성과변수:

        if var in df.columns:
            print(f" - 변수: {var}")
            stratified_test(subset, var)
        else:
            print(f" - {var} 없음")

📍 입지 그룹: 근접
 - 변수: log_방문자_리뷰수
   방법: Mann-Whitney U
   p-value: 3.6395e-02
   ✅ 동일 입지 내에서도 브랜드 차이 존재
 - 변수: log_블로그_리뷰수
   방법: Mann-Whitney U
   p-value: 5.9063e-07
   ✅ 동일 입지 내에서도 브랜드 차이 존재
 - 변수: log_월평균리뷰수
   방법: Mann-Whitney U
   p-value: 6.8263e-02
   ❌ 동일 입지 내 브랜드 차이 없음
📍 입지 그룹: 비근접
 - 변수: log_방문자_리뷰수
   방법: Mann-Whitney U
   p-value: 6.8572e-04
   ✅ 동일 입지 내에서도 브랜드 차이 존재
 - 변수: log_블로그_리뷰수
   방법: Mann-Whitney U
   p-value: 9.5259e-09
   ✅ 동일 입지 내에서도 브랜드 차이 존재
 - 변수: log_월평균리뷰수
   방법: Mann-Whitney U
   p-value: 1.0329e-03
   ✅ 동일 입지 내에서도 브랜드 차이 존재


### **출력결과**
**📍 입지 그룹: 근접**
- **변수: log_방문자_리뷰수**
- 방법: Mann-Whitney U
- p-value: 3.6395e-02
##### **✅ 동일 입지 내에서도 브랜드 차이 존재**
==================================================
- **변수: log_블로그_리뷰수**
- 방법: Mann-Whitney U
- p-value: 5.9063e-07
##### **✅ 동일 입지 내에서도 브랜드 차이 존재**
==================================================
- **변수: log_월평균리뷰수**
- 방법: Mann-Whitney U
- p-value: 6.8263e-02
##### **❌ 동일 입지 내 브랜드 차이 없음**
==================================================
#
**📍 입지 그룹: 비근접**
- **변수: log_방문자_리뷰수**
- 방법: Mann-Whitney U
- p-value: 6.8572e-04
##### **✅ 동일 입지 내에서도 브랜드 차이 존재**
==================================================
- **변수: log_블로그_리뷰수**
- 방법: Mann-Whitney U
- p-value: 9.5259e-09
##### **✅ 동일 입지 내에서도 브랜드 차이 존재**
==================================================
- **변수: log_월평균리뷰수**
- 방법: Mann-Whitney U
- p-value: 1.0329e-03
##### **✅ 동일 입지 내에서도 브랜드 차이 존재**
==================================================


### **5-4 자사 vs 경쟁사 입지 반응 구조 차이 검정**
- ***데이터 병합 → 결측·이상치 확인 → 로그 변환 → 분석용 데이터 저장***
- 공통 키 = place_id
- 리뷰 파일에 월평균리뷰수_log
- 입지 파일에 요리주점_밀도

| 단계               | 목적                  |
| ---------------- | ------------------- |
| 0. 파일 불러오기       | 리뷰/입지 데이터 읽기        |
| 1. 중복 확인         | place_id 유일성 확인     |
| 2. 병합 준비         | 컬럼 선택, 불필요 컬럼 제거    |
| 3. 병합            | LEFT JOIN           |
| 4. 결측 확인         | 매칭 성공/실패 비율         |
| 5. 타입 확인/변환      | TypeError 방지        |
| 6. 결측 처리         | NaN → 0             |
| 7. 이상치 확인        | 상위 percentile 확인    |
| 8. 로그 변환         | 왜도 완화, 분석용 컬럼 생성    |
| 9. 분석용 데이터 추출/저장 | 분석용 CSV 생성          |
| 10. 분포 확인        | zero-inflated 구조 확인 |

### **Step 1. 중복 확인**
- 두 데이터셋 불러오기 및 행 개수 확인

In [ ]:
import pandas as pd
import numpy as np

# 리뷰 데이터
review_path = r"C:\Users\user\Desktop\260218_데이터분석\수집 후 출력(CSV)\월평균_리뷰_최근1년_log.csv"
df_review = pd.read_csv(review_path)

# 입지 데이터
density_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\Step2_입지변수_결합완료.csv"
df_density = pd.read_csv(density_path)

print("리뷰 행:", len(df_review))
print("입지 행:", len(df_density))

리뷰 행: 430
입지 행: 430


### **Step 2. 병합 준비**
- 컬럼 폭증 방지
- 병합 직전에 불필요한 컬럼 제거

In [126]:
# 병합에 필요한 컬럼만 추출
df_density_sub = df_density[["place_id", "요리주점_밀도"]]

# 리뷰 파일에 이미 존재하면 제거
if "요리주점_밀도" in df_review.columns:
    df_review = df_review.drop(columns=["요리주점_밀도"])

### **Step 3. LEFT JOIN 병합**
- 리뷰 데이터를 기준으로 입지 변수 결합
- 결측 발생 시 리뷰 데이터는 남고, 매칭되지 않은 입지는 NaN

In [127]:
df_merged = pd.merge(
    df_review,
    df_density_sub,
    on="place_id",
    how="left"
)

print("병합 후 행:", len(df_merged))
print("병합 후 컬럼:", len(df_merged.columns))

병합 후 행: 430
병합 후 컬럼: 10


### **Step 4. 결측 확인 및 처리**
- NaN이 단순 결측인지, place_id 매칭 실패인지 확인

In [128]:
# NaN 비율 확인
print("요리주점_밀도 NaN 비율:", df_merged["요리주점_밀도"].isna().mean())

# 매칭 성공/실패 확인
matched = df_merged["요리주점_밀도"].notna().sum()
total = len(df_merged)
print("매칭 성공:", matched)
print("매칭 실패:", total - matched)

요리주점_밀도 NaN 비율: 0.29534883720930233
매칭 성공: 303
매칭 실패: 127


### **이상치 점검 전 수행하기**
- **NaN 비율 0.295 -> 약 29.5%**
- 단순한 결측이 아니라 place_id 매칭 실패 가능성
- **먼저 "요리주점 없음"인지 "병합 실패"인지 구분해야 함**
    1. 매칭 실패 기준 확인
    2. 이상치 점검 시 TypeError 원인 (순서대로)
    - TypeError: '<' not supported between instances of 'int' and 'str'
        - dtype 확인
        - 숫자로 강제 변환
        - 다시 확인

### **Step 5. 데이터 타입 확인 및 강제 변환**
- TypeError 방지
- 숫자로 변환하여 이상치/로그 변환 준비

In [129]:
# dtype 확인
print(df_merged["요리주점_밀도"].dtype)

# 숫자 아닌 값 NaN 처리 후 변환
df_merged["요리주점_밀도"] = pd.to_numeric(
    df_merged["요리주점_밀도"],
    errors="coerce"
)

print(df_merged["요리주점_밀도"].dtype)
print(df_merged["요리주점_밀도"].describe())

object
float64
count    302.000000
mean       0.089762
std        0.088083
min        0.000190
25%        0.005818
50%        0.063871
75%        0.162162
max        0.251004
Name: 요리주점_밀도, dtype: float64


### **Step 6. 실제 결측 처리**
- NaN → 0으로 처리하여, 요리주점이 없는 입지를 반영

In [131]:
# 실제 결측만 0으로 처리 (요리주점 없음 의미)
df_merged["요리주점_밀도"] = df_merged["요리주점_밀도"].fillna(0)

### **Step 7. 이상치 확인**
- 상위 극단값 확인
- 로그 변환 전 데이터 분포 확인

In [132]:
print(df_merged["요리주점_밀도"].describe())
print("상위 1% 컷:", df_merged["요리주점_밀도"].quantile(0.99))
print("상위 5% 컷:", df_merged["요리주점_밀도"].quantile(0.95))

count    430.000000
mean       0.063042
std        0.084452
min        0.000000
25%        0.000000
50%        0.007231
75%        0.144770
max        0.251004
Name: 요리주점_밀도, dtype: float64
상위 1% 컷: 0.250280445839022
상위 5% 컷: 0.2446483180428134


### **Step 8. 로그 변환**
- 왜도 완화
- 회귀 분석 안정화

In [133]:
# 로그 변환 (log1p)
df_merged["요리주점_밀도_log"] = np.log1p(df_merged["요리주점_밀도"])
df_merged["월평균_리뷰수_log"] = np.log1p(df_merged["월평균_리뷰수"])

# 혹시 중복된 log 컬럼 삭제
if "log_요리주점_밀도" in df_merged.columns:
    df_merged = df_merged.drop(columns=["log_요리주점_밀도"])

### **Step 9. 분석용 데이터 추출 및 저장**
- 회귀/통계 분석에 바로 사용 가능한 데이터셋 생성
- 분석용 파일 저장

In [134]:
analysis_cols = [
    "place_id",
    "월평균_리뷰수_log",
    "요리주점_밀도",
    "요리주점_밀도_log"
]

df_analysis = df_merged[analysis_cols]

save_path = r"C:\Users\BYB\Desktop\260218_데이터분석\분석용\merge파일.csv"
df_analysis.to_csv(save_path, index=False, encoding="utf-8-sig")
print("✅ 최종 분석 데이터 저장 완료")

✅ 최종 분석 데이터 저장 완료


### **Step 10. 분포 확인**
- zero-inflated 구조 확인
- 로그 변환 후 회귀 분석 준비

In [ ]:
print(df_merged["요리주점_밀도"].value_counts().head(10))
print("0 비율:", (df_merged["요리주점_밀도"] == 0).mean())

요리주점_밀도
0.000000    128
0.004760     20
0.019136     16
0.005818     16
0.144770     12
0.244399     12
0.007231     12
0.002720     11
0.004002     11
0.248509     10
Name: count, dtype: int64
0 비율: 0.29767441860465116


### **해석**
- 0값 (약 30%)
    - 해당 상권/반경에 요리주점 없음
    - NaN → 0 처리 후 포함됨
- 중간 값 (0~0.1 정도)
    - 낮은 밀도의 경쟁 존재
    - 대부분 소수점 단위로 분포
- 상위 값 (0.14~0.25)
    - 상권 내 요리주점 집중지역
    - 극단값으로 회귀 시 영향력 확인 필요

### **자사/경쟁사 파일 병합**

In [ ]:
# 분석용 merge 파일
merge_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\merge파일.csv"
df_analysis = pd.read_csv(merge_path)

# 자사/경쟁사 구분 파일
brand_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\Step1_전처리_자사경쟁사.csv"
df_brand = pd.read_csv(brand_path)

# 혹시 중복 place_id 있으면 제거
df_brand = df_brand.drop_duplicates(subset="place_id")

In [140]:
# place_id 기준 1:1 병합
df_analysis = pd.merge(
    df_analysis,
    df_brand[["place_id", "brand_type"]],
    on="place_id",
    how="left"
)

# 확인
print("병합 후 행 수:", len(df_analysis))
print("중복 place_id 여부:", df_analysis["place_id"].duplicated().sum())

병합 후 행 수: 430
중복 place_id 여부: 0


In [142]:
# 0/1 변수
df_analysis["요리주점_존재"] = (df_analysis["요리주점_밀도"] > 0).astype(int)

In [ ]:
save_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\merge_자사경쟁사.csv"
df_analysis.to_csv(save_path, index=False, encoding="utf-8-sig")
print("✅ 자사/경쟁사 그룹 포함 분석용 파일 저장 완료")

✅ 자사/경쟁사 그룹 포함 분석용 파일 저장 완료


### **5-4 시작**
- **동일 입지 변수에 대해 브랜드별 성과 반응이 다른지 검증**
- **핵심: “입지 변수 → 성과 변수 반응 차이”**
    - 입지 변수: 요리주점_밀도_log
    - 성과 변수: 월평균_리뷰수_log
    

1. 데이터 준비
    - merge_자사경쟁사.csv 불러오기
    - 필요한 컬럼: place_id, 월평균_리뷰수_log, 요리주점_밀도_log, brand_type, 요리주점_존재
2. 분석 변수 설정
    - 입지 변수: 요리주점_밀도_log
    - 성과 변수: 월평균_리뷰수_log
    - 그룹 구분: brand_type (자사/경쟁사)
3. 정규성 검정 포함
    - Shapiro-Wilk 검정 → Pearson/Spearman 자동 선택
4. 브랜드별 상관계수 계산
    - 상관계수 + p-value + 관찰치 수 + 분석 방법
5. 자사 vs 경쟁사 비교
    - pivot 테이블 생성 → 브랜드별 상관계수 비교
6. 결과 저장
    - CSV: Step5-4_상관분석결과.csv

In [ ]:
# 분석용 merge 파일
file_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\merge_자사경쟁사.csv"
df = pd.read_csv(file_path)

In [161]:
# 분석 변수 설정
density_col = "요리주점_밀도_log"      # 입지 변수
outcome_cols = ["월평균_리뷰수_log"]   # 분석 가능한 성과 변수
group_col = "brand_type"              # 자사/경쟁사 구분

In [169]:
# 정규성 검정 함수 (Shapiro-Wilk)
def check_normality(series):
    series = series.dropna()
    if len(series) < 3:
        return False  # 표본 너무 적으면 정규성 판단 불가
    stat, p = shapiro(series)
    return p > 0.05  # True면 정규성 만족

In [170]:
# 상관분석 함수
def calc_corr(x, y):
    # x, y 정규성 확인
    norm_x = check_normality(x)
    norm_y = check_normality(y)
    
    # 정규성 모두 만족 -> Pearson, 아니면 Spearman
    if norm_x and norm_y:
        corr = x.corr(y, method="pearson")
        method = "Pearson"
        # p-value는 scipy 사용
        from scipy.stats import pearsonr
        _, pval = pearsonr(x.dropna(), y.dropna())
    else:
        corr, pval = spearmanr(x.dropna(), y.dropna())
        method = "Spearman"
    return corr, pval, method


In [171]:
# 브랜드별 상관분석 수행
results = []

for brand in df[group_col].unique():
    df_brand = df[df[group_col] == brand]
    for outcome in outcome_cols:
        corr, pval, method = calc_corr(df_brand[density_col], df_brand[outcome])
        results.append({
            "브랜드": brand,
            "성과 변수": outcome,
            "상관계수": corr,
            "p-value": pval,
            "관찰치 수": len(df_brand),
            "분석 방법": method
        })

df_results = pd.DataFrame(results)

In [172]:
# 자사 vs 경쟁사 비교 (pivot)
df_pivot = df_results.pivot(index="성과 변수", columns="브랜드", values="상관계수").reset_index()

In [173]:
# 결과 확인
print("=== 브랜드별 요리주점 밀도 vs 성과 상관계수 ===")
print(df_results)

print("\n=== 자사 vs 경쟁사 상관계수 비교 ===")
print(df_pivot)

=== 브랜드별 요리주점 밀도 vs 성과 상관계수 ===
   브랜드        성과 변수      상관계수   p-value  관찰치 수     분석 방법
0  경쟁사  월평균_리뷰수_log  0.113752  0.039802    327  Spearman
1   자사  월평균_리뷰수_log  0.114229  0.250590    103  Spearman

=== 자사 vs 경쟁사 상관계수 비교 ===
브랜드        성과 변수       경쟁사        자사
0    월평균_리뷰수_log  0.113752  0.114229


### **해석**
- **상관계수 크기**
    - 자사와 경쟁사 모두 상관계수 ≈ 0.114 → **입지 민감도 비슷함**
    - 양수 → 요리주점 밀도가 높은 입지일수록 월평균 리뷰 수가 다소 증가하는 경향 있음
- **통계적 유의성**
    - 경쟁사 p = 0.04 → 유의미한 양의 상관관계
    - 자사 p = 0.25 → 통계적으로 유의하지 않음 (표본 수가 적은 영향 가능)
- **결론**
    - **전체적으로 입지 민감도(요리주점 밀도에 따른 리뷰 반응)는 자사와 경쟁사 유사**
    - 단, 자사 표본이 적어 통계적 검정력 낮음 → p-value 높게 나타남
    - 상권 집적 효과는 존재하나 강하지 않음

In [ ]:
# 결과 저장
save_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\Step5-4_상관분석결과.csv"
df_results.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"✅ Step5-4 결과 저장 완료: {save_path}")

✅ Step5-4 결과 저장 완료: C:\Users\BYB\Desktop\260218_데이터분석\분석용\Step5-4_상관분석결과.csv


### **5-5. 입지 효과의 통계적 차이 검정 (자사 vs 경쟁사)**
- **브랜드 차이 + 입지 그룹 차이를 동시에 검정**
    - (= 집단 평균 차이 + 구조적 차이)

In [5]:
file_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\merge_자사경쟁사.csv"
df = pd.read_csv(file_path)

In [6]:
# 2. 변수 설정
# ==============================
density_col = "요리주점_밀도_log"
outcome_col = "월평균_리뷰수_log"
group_col = "brand_type"

In [7]:
# 3. 요리주점 밀도 그룹화 (저/중/고)
# ==============================
df["요리주점_밀도_group"] = pd.qcut(
    df[density_col],
    q=3,
    labels=["저", "중", "고"]
)

In [ ]:
# 4. 브랜드 주효과 (Welch t-test)
# 자사 vs 경쟁사 평균 리뷰수 차이 검정
# ==============================
brand_groups = df[group_col].unique()

group1 = df[df[group_col] == brand_groups[0]][outcome_col]
group2 = df[df[group_col] == brand_groups[1]][outcome_col]

t_stat, p_brand = ttest_ind(group1, group2, equal_var=False)

print("=== 브랜드 주효과 (Welch t-test) ===")
print(f"t값: {t_stat}")
print(f"p-value: {p_brand}")

# t -> t-test -> 평균 차이 기반 계산

=== 브랜드 주효과 (Welch t-test) ===
t값: -4.166623809987936
p-value: 4.720068872408319e-05


### **해석**
- p < 0.05 (매우 작음)
    - **자사와 경쟁사의 평균 리뷰수는 통계적으로 유의하게 다르다**
- t가 음수라는 뜻은?
    - 첫 번째 그룹 평균 < 두 번째 그룹 평균
- **즉, 경쟁사 평균 리뷰수 > 자사 평균 리뷰수 (brand_type 정렬 순서에 따라 해석)**
#### **보고서용**
자사와 경쟁사의 월평균 리뷰수는 통계적으로 유의한 차이를 보였다 (t = -4.17, p < .001).
이는 브랜드 자체의 성과 수준 차이가 존재함을 의미한다.

In [ ]:
# 5. 밀도 주효과 (브랜드별 Welch ANOVA)
# 각 브랜드가 밀도(저·중·고)에 따라 차이 나는지 검정
# ==============================
results = []

for brand in brand_groups:
    df_brand = df[df[group_col] == brand]

    welch = pg.welch_anova(
        dv=outcome_col,
        between="요리주점_밀도_group",
        data=df_brand
    )

    results.append({
        "브랜드": brand,
        "F값": welch["F"].values[0],
        "p-value": welch["p-unc"].values[0]
    })

df_density = pd.DataFrame(results)

print("\n=== 밀도 주효과 (브랜드별 Welch ANOVA) ===")
print(df_density)


=== 밀도 주효과 (브랜드별 Welch ANOVA) ===
   브랜드        F값   p-value
0  경쟁사  2.143709  0.119762
1   자사  0.853019  0.431593


### **해석**
- 둘 다 p > 0.05
    - 요리주점 밀도(저·중·고)에 따라 리뷰수 차이 없음
- 즉, 밀도 구간화는 평균 성과 차이를 설명하지 못함
#### **보고서용**
요리주점 밀도 그룹(저·중·고)에 따른 평균 리뷰수 차이는 자사와 경쟁사 모두에서 통계적으로 유의하지 않았다.

### **브랜드 주효과 (Welch t-test) vs 밀도 주효과 (브랜드별 Welch ANOVA)**
| 검정          | 의미        |
| ----------- | --------- |
| t-test      | 브랜드 자체 차이 |
| Welch ANOVA | 밀도 영향 여부  |


In [14]:
# 6. 평균 패턴 확인 (구조 해석용)
# ==============================
mean_table = df.groupby(
    [group_col, "요리주점_밀도_group"]
)[outcome_col].mean().reset_index()

print("\n=== 브랜드 × 밀도 그룹 평균표 ===")
print(mean_table)


=== 브랜드 × 밀도 그룹 평균표 ===
  brand_type 요리주점_밀도_group  월평균_리뷰수_log
0        경쟁사             저     2.457918
1        경쟁사             중     2.401701
2        경쟁사             고     2.662788
3         자사             저     2.878682
4         자사             중     2.752292
5         자사             고     3.028717


C:\Users\user\AppData\Local\Temp\ipykernel_10044\3651454551.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_table = df.groupby(


In [18]:
# 7. 결과 저장
# ==============================
save_path = r"C:\Users\user\Desktop\260218_데이터분석\분석용\Step5-5_최종결과.csv"
df_results.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"\n✅ Step5-5 결과 저장 완료")


✅ Step5-5 결과 저장 완료


### **5-6. 동일 행정동 내 브랜드 집중도 효과 검정**
- 동일 행정동 내 자사 다점포 vs 단일점포 성과 차이 검정
- **✅ 집단 정의 (자사만 사용)**
    - 단일 점포 입점 → 해당 행정동 내 자사 점포 수 = 1
    - 다점포 입점 → 해당 행정동 내 자사 점포 수 ≥ 2
- **✅ 종속변수 (3개 사용)**
    - log_방문자리뷰수 (Step2 파일)
    - log_블로그리뷰수 (Step2 파일)
    - 월평균_리뷰수_log (merge 파일)
- **✅ 검정 방법**
    - 정규성 검정 (Shapiro)
    - 등분산 검정 (Levene)
    - 조건에 따라 자동 선택:
        - 정규 + 등분산 → t-test
        - 정규 + 등분산X → Welch t-test
        - 정규X → Mann-Whitney U

In [64]:
# 1. 데이터 로드
# =========================
file1 = r"C:\Users\user\Desktop\260218_데이터분석\분석용\Step2_입지변수_결합완료.csv"
file2 = r"C:\Users\user\Desktop\260218_데이터분석\분석용\merge_자사경쟁사.csv"

df1 = pd.read_csv(file1, encoding="utf-8-sig")
df2 = pd.read_csv(file2, encoding="utf-8-sig")

In [65]:
# 2. 월평균 리뷰수 merge
# ====================================
df2 = df2[["place_id", "월평균_리뷰수_log"]]
df = df1.merge(df2, on="place_id", how="left")

In [66]:
# 3. 금별맥주(자사)만 필터링
# ====================================
df_self = df[df["brand"] == "금별맥주"].copy()

print("금별맥주 총 점포 수:", len(df_self))

금별맥주 총 점포 수: 103


In [67]:
# 4. 동일 시군구 내 점포 수 계산
# ====================================
sigungu_count = (
    df_self.groupby("시군구명")
    .size()
    .reset_index(name="자사점포수")
)

df_self = df_self.merge(sigungu_count, on="시군구명", how="left")

In [73]:
# 단일 vs 다점포 구분
df_self["집중도"] = np.where(df_self["자사점포수"] == 1, "단일", "다점포")

print(df_self[["시군구명", "자사점포수", "집중도"]].head())
print("=" * 30)
print(df_self[["시군구명", "자사점포수", "집중도"]].tail())

   시군구명  자사점포수  집중도
0   구로구      3  다점포
1   강서구      7  다점포
2   송파구      2  다점포
3   강서구      7  다점포
4  서대문구      2  다점포
        시군구명  자사점포수  집중도
98       강서구      7  다점포
99       강서구      7  다점포
100      광명시      1   단일
101  안산시 상록구      1   단일
102  성남시 분당구      3  다점포


In [74]:
# 5. 분석 변수 설정
# ====================================
variables = [
    "log_방문자_리뷰수",
    "log_블로그_리뷰수",
    "월평균_리뷰수_log"
]

In [75]:
# 6. 집단 차이 검정
# ====================================
for var in variables:

    print(f"\n==============================")
    print(f"변수: {var}")
    print(f"==============================")

    data = df_self[[var, "집중도"]].dropna()

    group_single = data[data["집중도"] == "단일"][var]
    group_multi = data[data["집중도"] == "다점포"][var]

    print("단일 n:", len(group_single))
    print("다점포 n:", len(group_multi))

    if len(group_single) < 3 or len(group_multi) < 3:
        print("표본 부족 → 검정 불가")
        continue

    # 정규성
    p_norm1 = stats.shapiro(group_single)[1]
    p_norm2 = stats.shapiro(group_multi)[1]

    # 등분산성
    p_levene = stats.levene(group_single, group_multi)[1]

    print("정규성 p (단일):", p_norm1)
    print("정규성 p (다점포):", p_norm2)
    print("등분산성 p:", p_levene)

    # 검정 선택
    if p_norm1 > 0.05 and p_norm2 > 0.05:
        if p_levene > 0.05:
            t, p = stats.ttest_ind(group_single, group_multi, equal_var=True)
            print("→ Independent t-test")
        else:
            t, p = stats.ttest_ind(group_single, group_multi, equal_var=False)
            print("→ Welch t-test")
        print("t값:", t)
        print("p값:", p)
    else:
        u, p = stats.mannwhitneyu(group_single, group_multi)
        print("→ Mann-Whitney U")
        print("U값:", u)
        print("p값:", p)

    print("단일 평균:", group_single.mean())
    print("다점포 평균:", group_multi.mean())


변수: log_방문자_리뷰수
단일 n: 20
다점포 n: 83
정규성 p (단일): 0.968203143773934
정규성 p (다점포): 1.4133189653422342e-07
등분산성 p: 0.751994315157469
→ Mann-Whitney U
U값: 1110.0
p값: 0.019791815219236546
단일 평균: 5.916292724051404
다점포 평균: 5.36218345360536

변수: log_블로그_리뷰수
단일 n: 20
다점포 n: 83
정규성 p (단일): 0.25876573922698354
정규성 p (다점포): 0.0689676948780338
등분산성 p: 0.6370749634983983
→ Independent t-test
t값: 1.1625142018900751
p값: 0.24776617285662217
단일 평균: 4.356571681708934
다점포 평균: 4.017502749397069

변수: 월평균_리뷰수_log
단일 n: 20
다점포 n: 83
정규성 p (단일): 0.6338900675502834
정규성 p (다점포): 0.4089821304980622
등분산성 p: 0.6299362766485371
→ Independent t-test
t값: 2.7436256983028806
p값: 0.007191369634738533
단일 평균: 3.333282153694275
다점포 평균: 2.7928153658947474


### **해석**
#### ***log_방문자_리뷰수***
- 검정 선택 근거
    - 단일 정규성 p = 0.968 (>0.05)
    - 다점포 정규성 p < 0.001 → 정규성 위반
    - → Mann-Whitney U 사용

- 결과
    - U = 1110.0
    - p = 0.0198 (< 0.05)
    - 단일 평균 = 5.916
    - 다점포 평균 = 5.362

- 해석
    - 단일 입점 지역의 방문자 리뷰 수가 통계적으로 유의하게 높음
    - 동일 시군구 내 다점포 전략은 방문자 리뷰 수 측면에서 성과 우위가 없음
    - 오히려 단일 입점이 더 높은 반응을 보임

#### **log_블로그_리뷰수**
- 검정 선택 근거
    - 두 집단 모두 정규성 충족
    - 등분산성 충족
    - → Independent t-test

- 결과
    - t = 1.16
    - p = 0.248 (> 0.05)
    - 단일 평균 = 4.357
    - 다점포 평균 = 4.018

- 해석
    - 블로그 리뷰 수는 집단 간 유의한 차이 없음
    - 브랜드 집중도는 블로그 확산 효과에 영향 없음

#### **월평균_리뷰수_log**
- 검정 선택 근거
    - 정규성 충족
    - 등분산성 충족
    - → Independent t-test

- 결과
    - t = 2.744
    - p = 0.0072 (< 0.01)
    - 단일 평균 = 3.333
    - 다점포 평균 = 2.793

- 해석
    - 단일 입점 지역의 월평균 리뷰 수가 유의하게 높음
    - 다점포 전략은 월간 리뷰 활성도 측면에서 성과 감소와 연관

| 변수     | 결과            | 해석       |
| ------ | ------------- | -------- |
| 방문자 리뷰 | 유의 (p=0.0198) | 단일 > 다점포 |
| 블로그 리뷰 | 비유의           | 차이 없음    |
| 월평균 리뷰 | 유의 (p=0.007)  | 단일 > 다점포 |

- 동일 시군구 내에서 금별맥주가 다점포로 확장할 경우,
- 방문자 리뷰 및 월간 리뷰 활성도는 오히려 감소하는 경향을 보였다.

- 즉, 지역 내 브랜드 집중 전략은 온라인 반응 지표 측면에서 긍정적 효과를 보이지 않는다.
